# Paralelni algoritmi - treći projekat

- Projekat se radi individualno ili u paru

- Rok za predaju projekata je ponedeljak 28.12.2020. Biće potrebno da se studenti koji planiraju da urade i odbrane projekat prijave, do datuma i procedurom koji će naknadno biti objavljeni.

- Predaja projekata je putem emaila, programski kod bi trebalo da
bude prikacen uz sam email. Link ka Collab-u je opcion. Ukoliko dva
studenta rade u paru, dovoljno je da jedan od njih preda, pri cemu
drugog stavi u cc. E-mail treba da sadrzi sve podatke (ime, prezime,
broj indeksa) za oba studenta.

Koristeći PyCuda okurženje napisati CUDA program za (matrično) množenje matrica.

1. [5 bodova] Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)
2. [5 bodova] Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. [7 bodova] Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)
4. [8 bodova] Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 16.4MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620989 sha256=8c716a1e99d7e2eabfd28b9024f78633ba3ab035a46613a76f1035c3a550b585
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.4-py2.py3-none-any.whl size=59109 sha256=447935b7cdd575d314f44cb93d190b1303bae6ec5c4848a120ddec7ee009b4b1
  Stored in directory: /root/.cache/pip/wheels/6f/2a/6e/0b210d900f2b6caca133395d2fa153d8a9c03ad42c3c5d55bf
Successfully built pycuda pytools


In [ ]:
!nvidia-smi

Fri Dec 18 14:00:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ********************************************* PRVA TACKA **********************************************

import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(
    """ 
        __global__ void mnozenje (float *a, float *b, float *c, int sirinaA, int sirinaB){
              int idxC = threadIdx.x + threadIdx.y * sirinaB;
              int idxA = threadIdx.y * sirinaA;
              int idxB = threadIdx.x;
              
              for(int i = 0; i < sirinaA; i += 1){
                  c[idxC] += a[idxA + i] * b[idxB];
                  idxB += sirinaB;
              }
          }
    """
)

a = np.random.randn(20,10).astype(dtype=np.float32)
b = np.random.randn(10,15).astype(dtype=np.float32)

ocekivano = np.matmul(a, b)

c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

func = mod.get_function("mnozenje")
func(a_gpu, b_gpu, c_gpu, np.int32(a.shape[1]), np.int32(b.shape[1]), block = (b.shape[1], a.shape[0], 1), grid = (1, 1, 1))

cuda.memcpy_dtoh(c,c_gpu)

print("Dobijen ocekivan rezultat: ", (c == ocekivano).all())

Dobijen ocekivan rezultat:  True


In [ ]:
# ********************************************* DRUGA TACKA **********************************************

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

mod = SourceModule(
    """ 
        __global__ void mnozenje (float *a ,float *b, float *c, int sirinaB, int visinaA, int sirinaA){
                long idxC = threadIdx.x + blockDim.x * blockIdx.x + threadIdx.y * sirinaB + blockDim.y * blockIdx.y * sirinaB;
                long idxA = (threadIdx.y + blockDim.y * blockIdx.y) * sirinaA;
                long idxB = blockDim.x * blockIdx.x + threadIdx.x;

                if(idxB > sirinaB) return;
                if(threadIdx.y + blockDim.y * blockIdx.y > visinaA) return;
              
                for(int i = 0; i < sirinaA; i += 1){
                    c[idxC] += a[idxA + i] * b[idxB];
                    idxB += sirinaB;
                }
            }
    """
)

a = np.random.randn(90,64).astype(dtype=np.float32)
b = np.random.randn(64,50).astype(dtype=np.float32)

ocekivano = np.matmul(a, b)

c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu,a)
cuda.memcpy_htod(b_gpu,b)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("mnozenje")
func(a_gpu, b_gpu, c_gpu, np.int32(b.shape[1]), np.int32(a.shape[0]), np.int32(a.shape[1]), block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
# print(ocekivano)
# print("\n")
# print(c)

print("Dobijen ocekivan rezultat: ", (c == ocekivano).all())

Dobijen ocekivan rezultat:  True


In [ ]:
# ********************************************* TRECA TACKA **********************************************

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

mod = SourceModule(
    """ 
        __global__ void mnozenje(float *a, float *b, float *c, int visinaA, int sirinaA, int visinaB, int sirinaB, int visinaC, int sirinaC){
                float novaVrednost = 0;

                int red = blockIdx.y * blockDim.y + threadIdx.y;
                int kolona = blockIdx.x * blockDim.x + threadIdx.x;

                __shared__ float deljenaA[32][32];
                __shared__ float deljenaB[32][32];

                for(int i = 0; i < sirinaA / blockDim.x; i += 1){
                    if(i * blockDim.x + threadIdx.x < sirinaA && red < visinaA){
                        deljenaA[threadIdx.y][threadIdx.x] = a[red * sirinaA + i * blockDim.y + threadIdx.x];
                    } else {
                        deljenaA[threadIdx.y][threadIdx.x] = 0.0;
                    }

                    if(i * blockDim.y + threadIdx.y < visinaB && kolona < sirinaB){
                        deljenaB[threadIdx.y][threadIdx.x] = b[(i * blockDim.y + threadIdx.y) * sirinaB + kolona];
                    } else {
                        deljenaB[threadIdx.y][threadIdx.x] = 0.0;
                    }

                    __syncthreads();

                    for(int j = 0; j < blockDim.x; j += 1){
                        novaVrednost += deljenaA[threadIdx.y][j] * deljenaB[j][threadIdx.x];
                    }

                    __syncthreads();

                    if(red < visinaC && kolona < sirinaC){
                        c[(blockIdx.y * blockDim.y + threadIdx.y) * sirinaC + blockIdx.x * blockDim.x + threadIdx.x] = novaVrednost;
                    }                
                } 
            }
    """
)

a = np.random.randn(100,64).astype(dtype=np.float32)
b = np.random.randn(64,70).astype(dtype=np.float32)

ocekivano = np.matmul(a, b)

c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu,a)
cuda.memcpy_htod(b_gpu,b)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("mnozenje")
func(a_gpu, b_gpu, c_gpu, np.int32(a.shape[0]), np.int32(a.shape[1]), np.int32(b.shape[0]), np.int32(b.shape[1]),
     np.int32(a.shape[0]),np.int32(b.shape[1]), block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)

print("Dobijen ocekivan rezultat: ", (c == ocekivano).all())

Dobijen ocekivan rezultat:  True


In [ ]:
# ********************************************* CETVRTA TACKA **********************************************

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

mod = SourceModule(
    """ 
        __global__ void mnozenje(float *a, float *b, float *c, int redoviA, int sirinaA, int redoviB, int sirinaB, int redoviC, int sirinaC){
                float novaVrednost = 0;

                int red = blockIdx.y * blockDim.y + threadIdx.y;
                int kolona = blockIdx.x * blockDim.x + threadIdx.x;

                __shared__ float deljenaA[32][32];
                __shared__ float deljenaB[32][32];

                  for(int i = 0; i <= (sirinaA - 1) / blockDim.x; i += 1){
                      if(i * blockDim.x + threadIdx.x < sirinaA && red < redoviA){
                          deljenaA[threadIdx.y][threadIdx.x] = a[red * sirinaA + i * blockDim.x + threadIdx.x];
                      } else {
                          deljenaA[threadIdx.y][threadIdx.x] = 0.0;
                      }

                      if(i * blockDim.y + threadIdx.y < sirinaB && kolona < redoviB){
                          deljenaB[threadIdx.y][threadIdx.x] = b[(i * blockDim.y + threadIdx.y) + sirinaB * kolona];
                      } else {
                          deljenaB[threadIdx.y][threadIdx.x] = 0.0;
                      }

                      __syncthreads();

                      for(int j = 0; j < blockDim.y; j += 1){
                          novaVrednost += deljenaA[threadIdx.y][j] * deljenaB[j][threadIdx.x];
                      }

                      __syncthreads();

                      if(red < redoviC && kolona < sirinaC){
                          c[red * sirinaC + kolona] = novaVrednost;
                      }                 
                } 
            }
    """
)

a = np.random.randn(40,64).astype(dtype=np.float32)
b = np.random.randn(90,64).astype(dtype=np.float32)

redoviA = np.int32(a.shape[0])
redoviB = np.int32(b.shape[0])

ocekivano = np.matmul(a, np.transpose(b))

c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu,a)
cuda.memcpy_htod(b_gpu,b)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("mnozenje")
func(a_gpu, b_gpu, c_gpu, redoviA, np.int32(a.shape[1]), redoviB, np.int32(b.shape[1]), redoviA, redoviB, block = (32, 32, 1), grid=(math.ceil(redoviB/32), math.ceil(redoviA/32), 1))

cuda.memcpy_dtoh(c, c_gpu)

print("Dobijen ocekivan rezultat: ", (c == ocekivano).all())

Dobijen ocekivan rezultat:  True
